<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/%E8%BC%95%E6%B2%B9%E6%A1%88%E5%AD%90_%E4%BA%A4%E5%8F%89%E9%A9%97%E8%AD%89_%E9%81%B8%E6%A8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import warnings;warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate
from sklearn.linear_model import BayesianRidge, LinearRegression,Lasso,Ridge
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,max_error,make_scorer
from math import sqrt
from sklearn.manifold import isomap

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# some function

In [0]:
def rmse(y,y_pred):
  return sqrt(mean_squared_error(y,y_pred))
rmse_scorer = make_scorer(rmse)

In [0]:
def metric(y_t,y_p,name):
  res = {'R2':r2_score(y_t,y_p),
         'RMSE':sqrt(mean_squared_error(y_t,y_p)),
         'mean_absolute_error':mean_absolute_error(y_t,y_p),
         'mean_absolute_error/y_mean' : mean_absolute_error(y_t,y_p) / np.mean(y_t),
         'RMSE/y_mean' : sqrt(mean_squared_error(y_t,y_p)) / np.mean(y_t),
         'y_mean' : np.mean(y_t),
         'std' : np.std(y_t),
         }
  return pd.DataFrame(res,index=[name])

In [0]:
def modeling(model,train,test,features_map):
  res = pd.DataFrame()
  for y_name in features_map.keys():
    model.fit(train[features_map[y_name]],train[y_name])
    y_pred = model.predict(test[features_map[y_name]])
    y_pred = pd.DataFrame(y_pred,columns=[y_name])
    res = res.append(metric(test[y_name],y_pred[y_name],y_name))
  res.loc['AVG'] = res.mean()
  return res

# load

In [5]:
data = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/test_250.csv',index_col=0)
train = data.loc[data.index %5 != 0]
test = data.loc[data.index %5 == 0]
test = test.drop(95,axis=0)
print(train.shape)
print(test.shape)

(200, 27)
(49, 27)


# 定義欄位

In [0]:
cols = train.columns.tolist()
features = cols[:4]
y_columns = cols[4:]

# 特徵縮放

In [0]:
scaler_x = StandardScaler()
train[features] = scaler_x.fit_transform(train[features])
test[features] = scaler_x.transform(test[features])

# 降維萃取特徵

In [8]:
iso = isomap.Isomap(n_components=1)
train['f1'] = iso.fit_transform(train[['T10','N+A']]) 
test['f1'] = iso.transform(test[['T10','N+A']]) 
features = features + ['f1']
features

['T10', 'T50', 'T90', 'N+A', 'f1']

In [9]:
iso = isomap.Isomap(n_components=1)
train['f2'] = iso.fit_transform(train[['T50','N+A']]) 
test['f2'] = iso.transform(test[['T50','N+A']])
features = features + ['f2']
features

['T10', 'T50', 'T90', 'N+A', 'f1', 'f2']

In [10]:
iso = isomap.Isomap(n_components=1)
train['f3'] = iso.fit_transform(train[['T90','N+A']]) 
test['f3'] = iso.transform(test[['T90','N+A']]) 
features = features + ['f3']
features

['T10', 'T50', 'T90', 'N+A', 'f1', 'f2', 'f3']

In [11]:
print(features)
print(y_columns)

['T10', 'T50', 'T90', 'N+A', 'f1', 'f2', 'f3']
['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP', 'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A']


In [12]:
res = pd.DataFrame()
features_map = {}
for y_name in y_columns:
  corrs = abs(train[features+[y_name]].corr())[[y_name]].drop(y_name).sort_values(by=y_name,ascending=False)
  best_feature = corrs.head(1).index[0]
  if best_feature not in ['T10','T50','T90','N+A']:
    res.loc[y_name,'features'] = str(['T10','T50','T90','N+A',best_feature])
    features_map[y_name] =  ['T10','T50','T90','N+A',best_feature]
  else:
    res.loc[y_name,'features'] = str(['T10','T50','T90','N+A'])
    features_map[y_name] =  ['T10','T50','T90','N+A']
res

,features
C5NP,"['T10', 'T50', 'T90', 'N+A']"
C5IP,"['T10', 'T50', 'T90', 'N+A']"
C5N,"['T10', 'T50', 'T90', 'N+A']"
C6NP,"['T10', 'T50', 'T90', 'N+A', 'f2']"
C6IP,"['T10', 'T50', 'T90', 'N+A']"
C6N,"['T10', 'T50', 'T90', 'N+A']"
C6A,"['T10', 'T50', 'T90', 'N+A']"
C7NP,"['T10', 'T50', 'T90', 'N+A']"
C7IP,"['T10', 'T50', 'T90', 'N+A']"
C7N,"['T10', 'T50', 'T90', 'N+A', 'f3']"


# 交叉驗證選模

In [13]:
model = CatBoostRegressor(learning_rate = 0.1,n_estimators=100,verbose=0)
res = pd.DataFrame()
for y_name in features_map.keys():
  cv = cross_validate(model,train[features_map[y_name]],train[y_name],cv=5,scoring=rmse_scorer)
  res.loc[y_name,'五次交叉驗證_RMSE'] = str(cv['test_score'])
  res.loc[y_name,'RMSE_AVG'] = cv['test_score'].mean()
res.loc['AVG'] = res.mean()
res.style.background_gradient()

,五次交叉驗證_RMSE,RMSE_AVG
C5NP,[1.01818665 1.68638133 1.25374241 0.73938986 0.61294555],1.06213
C5IP,[1.00493015 1.305182 0.65496775 0.423213 0.49831917],0.777322
C5N,[0.1020321 0.18627511 0.09171565 0.07096808 0.10852262],0.111903
C6NP,[0.68478189 0.95415479 0.9336779 0.55369603 0.74284143],0.77383
C6IP,[0.74242126 1.27246 0.87011321 0.7034917 0.71364666],0.860427
C6N,[0.88233769 1.55967409 1.36701708 1.35865381 1.22319488],1.27818
C6A,[0.92107804 1.25649531 1.1677622 0.27730776 0.29253778],0.783036
C7NP,[0.88101601 0.9183829 0.63455965 0.79878795 0.55915307],0.75838
C7IP,[0.99734245 0.87040428 1.15856693 0.5767496 0.79181151],0.878975
C7N,[2.17174659 2.51500738 2.6056954 1.79261074 1.7218674 ],2.16139


In [14]:
model = SVR()
res = pd.DataFrame()
for y_name in features_map.keys():
  cv = cross_validate(model,train[features_map[y_name]],train[y_name],cv=5,scoring=rmse_scorer)
  res.loc[y_name,'五次交叉驗證_RMSE'] = str(cv['test_score'])
  res.loc[y_name,'RMSE_AVG'] = cv['test_score'].mean()
res.loc['AVG'] = res.mean()
res.style.background_gradient()

,五次交叉驗證_RMSE,RMSE_AVG
C5NP,[1.35650171 2.01090279 1.54045921 0.79063067 0.63959126],1.26762
C5IP,[1.24040863 1.6185643 0.75768913 0.47107689 0.57473305],0.932494
C5N,[0.10396525 0.20293397 0.11558675 0.0893688 0.10198732],0.122768
C6NP,[0.84916926 1.17089601 0.97813089 0.69326424 0.89545171],0.917382
C6IP,[0.80574263 1.28076607 0.85710159 0.74026627 0.70191406],0.877158
C6N,[1.13173872 1.64537419 1.3135155 1.6635868 1.35815981],1.42248
C6A,[0.99277633 0.77361613 1.22126915 0.22394406 0.25247885],0.692817
C7NP,[0.74537005 0.8587616 0.64377504 0.70569239 0.49404051],0.689528
C7IP,[1.0251193 0.90652457 1.37768275 0.79254192 0.74139005],0.968652
C7N,[2.22819355 3.16128679 3.62897021 2.11222139 1.75353674],2.57684


In [15]:
model = LinearRegression()
res = pd.DataFrame()
for y_name in features_map.keys():
  cv = cross_validate(model,train[features_map[y_name]],train[y_name],cv=5,scoring=rmse_scorer)
  res.loc[y_name,'五次交叉驗證_RMSE'] = str(cv['test_score'])
  res.loc[y_name,'RMSE_AVG'] = cv['test_score'].mean()
res.loc['AVG'] = res.mean()
res.style.background_gradient()

,五次交叉驗證_RMSE,RMSE_AVG
C5NP,[1.56590169 1.96226684 1.45293262 0.82623201 0.79196836],1.31986
C5IP,[1.37211008 1.53709956 0.93860852 0.79361366 0.75325822],1.07894
C5N,[0.09217735 0.17010235 0.09915038 0.09156337 0.11461359],0.113521
C6NP,[1.10517499 1.06873016 1.04366067 0.85408832 0.94155239],1.00264
C6IP,[0.56564473 1.03706818 1.00842759 0.80814363 0.79118623],0.842094
C6N,[1.12994739 1.82954048 1.26823988 1.39336521 1.32519207],1.38926
C6A,[1.12453121 1.04800528 1.32198882 0.48471355 0.29811775],0.855471
C7NP,[0.96766573 1.17308937 0.82667408 0.86246054 0.57186117],0.88035
C7IP,[1.5627688 1.72693929 1.43895428 1.48273652 0.99359323],1.441
C7N,[2.28223676 3.32361334 3.29890426 2.57181963 1.6919933 ],2.63371


# 測試49筆

In [16]:
model = CatBoostRegressor(learning_rate = 0.1,n_estimators=100,verbose=0)
modeling(model,train,test,features_map).style.background_gradient()

,R2,RMSE,mean_absolute_error,mean_absolute_error/y_mean,RMSE/y_mean,y_mean,std
C5NP,0.97502,0.621353,0.45645,0.112244,0.152794,4.0666,3.93137
C5IP,0.974611,0.476763,0.336502,0.122437,0.173472,2.74836,2.99215
C5N,0.949621,0.0708534,0.0508445,0.110467,0.153939,0.460268,0.315672
C6NP,0.882341,0.671262,0.49003,0.12512,0.171394,3.91649,1.95695
C6IP,0.910867,0.789889,0.4977,0.115836,0.183841,4.29659,2.64574
C6N,0.783154,0.848247,0.569959,0.0946346,0.140841,6.02273,1.82157
C6A,0.729212,0.464509,0.300653,0.309234,0.477767,0.97225,0.892646
C7NP,0.758624,0.414156,0.290438,0.0532388,0.0759168,5.45539,0.842978
C7IP,0.666713,0.908459,0.578041,0.102036,0.160361,5.66509,1.57361
C7N,0.948986,1.32102,1.0012,0.0636486,0.0839799,15.7301,5.84872
